 # DA Final project:

# Group 52

# India Agriculture Crop Production

###  Name of students:
Koliya Aharon<br>
Ateret Bass

![indiaagriculture](./india.PNG)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from IPython.display import Image

In [2]:
import json
with open("Indian_States.json") as f:
    states = json.load(f)

# Data Preparation

In [3]:
df = pd.read_csv("data/India Agriculture Crop Production.csv")

In [4]:
df.shape

(345407, 10)

In [5]:
df.columns

Index(['State', 'District', 'Crop', 'Year', 'Season', 'Area', 'Area Units',
       'Production', 'Production Units', 'Yield'],
      dtype='object')

In [6]:
df.dtypes

State                object
District             object
Crop                 object
Year                 object
Season               object
Area                float64
Area Units           object
Production          float64
Production Units     object
Yield               float64
dtype: object

In [7]:
df.isnull().sum().sort_values()

State                  0
District               0
Year                   0
Area Units             0
Production Units       0
Season                 1
Crop                  32
Area                  33
Yield                 33
Production          4993
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345407 entries, 0 to 345406
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   State             345407 non-null  object 
 1   District          345407 non-null  object 
 2   Crop              345375 non-null  object 
 3   Year              345407 non-null  object 
 4   Season            345406 non-null  object 
 5   Area              345374 non-null  float64
 6   Area Units        345407 non-null  object 
 7   Production        340414 non-null  float64
 8   Production Units  345407 non-null  object 
 9   Yield             345374 non-null  float64
dtypes: float64(3), object(7)
memory usage: 26.4+ MB


In [9]:
df = df.dropna()

In [10]:
def change_year(year):
    first_half = int(year.split("-")[0])
    return str(first_half) + "-" + str(first_half + 1)

In [11]:
df.Year = df.Year.apply(change_year)

In [12]:
df = df.sort_values("Year").reset_index(drop = True)

In [13]:
df.head()

,State,District,Crop,Year,Season,Area,Area Units,Production,Production Units,Yield
0,Meghalaya,SOUTH GARO HILLS,Rapeseed &Mustard,1997-1998,Rabi,257.0,Hectare,176.0,Tonnes,0.684825
1,Uttar Pradesh,CHANDAULI,Jowar,1997-1998,Kharif,958.0,Hectare,929.0,Tonnes,0.969729
2,Bihar,DARBHANGA,Arhar/Tur,1997-1998,Kharif,1017.0,Hectare,1665.0,Tonnes,1.637168
3,Uttar Pradesh,BULANDSHAHR,Jowar,1997-1998,Kharif,487.0,Hectare,386.0,Tonnes,0.792608
4,Uttar Pradesh,BUDAUN,Jowar,1997-1998,Kharif,5164.0,Hectare,5435.0,Tonnes,1.052479


In [14]:
df.nunique()

State                   36
District               728
Crop                    56
Year                    24
Season                   6
Area                 47949
Area Units               1
Production           69078
Production Units         3
Yield               197154
dtype: int64

In [15]:
df.describe()

,Area,Production,Yield
count,3.404140e+05,3.404140e+05,340414.000000
mean,1.182535e+04,9.583711e+05,80.564577
std,4.612484e+04,2.152986e+07,923.232000
min,8.000000e-03,0.000000e+00,0.000000
25%,8.000000e+01,8.700000e+01,0.567920
50%,5.630000e+02,7.170000e+02,1.025000
75%,4.257000e+03,7.176000e+03,2.500000
max,8.580100e+06,1.597800e+09,43958.333333


# Crops efficiency

To define efficiency we need to look at the `Yield` column.

Since `Yield` is definde as: 
$$
\text{Yield} = \frac{\text{Production}}{\text{Area}}
$$
Meaning that we calculate how much Crops one unit of `Area` (Hectare) outputs.

For comparision we will look only at crops that the `Production Unit` is `Tonnes`.

In [16]:
Crop_efficiency_data = df[df["Production Units"] == "Tonnes"]

## Basic Crop efficiency

<div dir="rtl">
מהו היבול הכי יעיל ?
</div>

In [ ]:
crop_efficiency = Crop_efficiency_data.groupby("Crop").Yield.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(crop_efficiency, y= "Crop", x= "Yield", title= "Crop average efficiency")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

<div dir="rtl">
היבול הכי יעיל בממוצע הוא קנה סוכר
</div>

## Crop quantity

<div dir="rtl">
מהו היבול עם התפוקה הגבוהה ביותר ?
</div>

In [ ]:
crop_quantity = Crop_efficiency_data.groupby("Crop").Production.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(crop_quantity, y= "Crop", x= "Production", title= "Crop average quantity")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

<div dir="rtl">
היבול עם הכי הרבה תפוקה בממוצע הוא גם קנה סוכר
</div>

![indiaagriculture](./sugar.PNG)

# Crop Area

<div dir="rtl">
שאלה: האם ליבול היעיל ביותר בממוצע עם התפוקה הגבוהה ביותר בממוצע מוקצה הכי הרבה שטח בממוצע ?
</div>

In [ ]:
crop_area = Crop_efficiency_data.groupby("Crop").Area.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(crop_area, y= "Crop", x= "Area", title= "Crop average area")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

<div dir="rtl">
מסקנה: למרות שקנה סוכר הוא היבול היעיל ביותר בממוצע עם התפוקה הגדולה ביותר בממוצע לא מוקצה לו הכי הרבה שטח בממוצע.
    במקום זה ניתן לראות שדווקא הוקצה הכי הרבה שטח לזרעים לשמן, חיטה ואורז.
    שהם גידוליים בסיסיים לחיי אדם ולכן להם הוקצה את רוב השטח. 
    בנוסף אנחנו רואים שקנה סוכר לא נמצא אפילו בעשירייה הראשונה ועדיין הוא מייצר הכי הרבה תפוקה.
    מה שמחזק את רמת היעילות שלו.
</div>

## Crop efficiency by State

In [ ]:
state_most_efficient_crop = Crop_efficiency_data.groupby(["State"]).apply(lambda x: x.groupby("Crop").Yield.mean().idxmax())
state_most_efficient_crop = state_most_efficient_crop.to_frame().reset_index().rename({0:"Crop"}, axis= 1)

In [ ]:
px.choropleth_mapbox(state_most_efficient_crop, geojson= states,featureidkey= "properties.NAME_1", locations="State",
                     color= "Crop",
                        color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 22.736424, "lon":78.473622},
                           opacity=0.5)

<div dir="rtl">
העשרה: ניתן לראות שקני סוכר הוא גם היבול היעיל ביותר ברוב המדינות
</div>

In [ ]:
del crop_efficiency
del crop_quantity
del crop_area

# States

## State efficiency

<div dir="rtl">
שאלה: מה המדינה שמקצה הכי הרבה שטח ליבול בממוצע ?
</div>

In [ ]:
state_area = Crop_efficiency_data.groupby("State").Area.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(state_area, y= "State", x= "Area", title= "Crop average area")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

In [ ]:
state_area.sort_values("Area", ascending= False).head()

<div dir="rtl">
הטבלה מציגה את חמש המדינות שמקצות הכי הרבה שטח ליבול בממוצע
</div>

In [ ]:
del state_area

<div dir="rtl">
השאלה : האם המדינה שמקצה בממוצע הכי הרבה שטח ליבול היא המדינה הכי יעילה בממוצע ?
</div>

In [ ]:
mean_crop_efficiency_by_state =  Crop_efficiency_data.groupby(["State"]).Yield.mean().to_frame().reset_index()

In [ ]:
px.choropleth_mapbox(mean_crop_efficiency_by_state, geojson= states,featureidkey= "properties.NAME_1", locations="State",
                     color= "Yield",
                        color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 22.736424, "lon":78.473622},
                           opacity=0.5)

In [ ]:
mean_crop_efficiency_by_state.sort_values("Yield", ascending=False).head()

<div dir="rtl">
הטבלה מציגה את חמש המדינות היעלות ביותר בממוצע
</div>

<div dir="rtl">
מסקנה: חמש המדינות היעילות ביותר בממוצע ליבול הן לא המדינות המקצות את ממוצע השטח הגדול ביותר וניתן להבין שאין קשר בין גודל השטח ליעילות שלו 
</div> 

![indiaagriculture](./ef.PNG)

## State efficiency by Season

<div dir="rtl">
שאלה: האם יש קשר בין המיקום הגיאוגרפי לעונה היעילה ביותר בכל מדינה ?
</div>

In [ ]:
state_most_efficient_season = Crop_efficiency_data[Crop_efficiency_data != "Whole Year"].groupby(["State"]).apply(lambda x: x.groupby("Season").Yield.mean().idxmax())
state_most_efficient_season = state_most_efficient_season.to_frame().reset_index().rename({0:"Season"}, axis= 1)

In [ ]:
px.choropleth_mapbox(state_most_efficient_season, geojson= states,featureidkey= "properties.NAME_1", locations="State",
                     color= "Season",
                        color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 22.736424, "lon":78.473622},
                           opacity=0.5)

<div dir="rtl">
מסקנה: ישנו קשר בין המיקום הגיאוגרפי לעונה היעילה ביותר ליבול בכל מדינה ניתן לראות בגרף שהחלק המערבי יעיל בעונה רבי החלק המרכזי יעיל בקיץ והמזרחי יעיל בחורף 
</div>

![indiaagriculture](./se.PNG)

In [ ]:
px.imshow(Crop_efficiency_data[Crop_efficiency_data.Season != "Whole Year"]\
          .pivot_table(index= "Year", columns= "Season", values= "Yield").corr(), text_auto=True)